<a href="https://colab.research.google.com/github/ryali93/utilitarios/blob/master/agol/AGOL2SHP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fiona
!pip install geopandas

In [ ]:
import os, requests
import json, uuid
import pandas as pd
import geopandas as gpd
import traceback

nombreSHP = "puntos_delitos"
url = 'https://arcgis3.inei.gob.pe:6443/arcgis/rest/services/Datacrim/DATACRIM002_AGS_PUNTOSDELITOS/MapServer/146/query'

def convert_to_geojson(pjson_feature):
    geom_coords = list(pjson_feature['geometry'].values())
    geojson_geom = {
        "type": "Point",
        "coordinates": geom_coords
    }

    return {
        "type": "Feature",
        "geometry": geojson_geom,
        "properties": pjson_feature['attributes']
    }

class RequestDataAGOL(object):
    def __init__(self):
        self.url = url
        self.listaOids = []

    def countOnly(self):
        response = requests.post(
            self.url,
            data = {
            'where': '1=1',
            'outFields': "*",
            'returnCountOnly': 'true',
            'f': 'pjson'
            }
        )
        res = json.loads(response.text)
        self.cantidad = res["count"]
        print(self.cantidad)

    def oidsOnly(self):
        response = requests.post(
            self.url,
            data = {
            'where': "1=1",
            'outFields': "*",
            'returnIdsOnly': 'true',
            'f': 'pjson'
            }
        )
        res = json.loads(response.text)
        self.listaOids.extend(res["objectIds"])
        self.listaOids = list(set(self.listaOids))

    def requestData(self):
        global nombreSHP
        data = self.listaOids
        chunks = [data[x:x+1000] for x in range(0, len(data), 1000)]
        geo_df = ""
        a = 0
        for n in chunks:
          a += 1
          try:
            print(len(n))
            n = ','.join([str(x) for x in n])
            response = requests.post(
                self.url,
                data = {
                'objectIds': n,
                'outFields': "*",
                'f': 'json'
                }
            )
            res = json.loads(response.text)
            features =  [convert_to_geojson(x) for x in res['features'] if 'geometry' in x.keys()]

            gdf = gpd.GeoDataFrame.from_features(features)
            gdf.crs = {'init' :'epsg:4326'}
            if type(geo_df) == str:
              geo_df = gdf[:]
            else:
              # geo_df = geo_df.append(gdf)
              geo_df = gpd.GeoDataFrame(pd.concat([geo_df, gdf], ignore_index=True))

            print("geo_df tiene {} elementos".format(len(geo_df)))

            geo_df.to_file(driver="ESRI Shapefile", filename="{}.shp".format(nombreSHP))
          except:
            nombreSHP = "{}_{}".format(nombreSHP, a)
            traceback.print_exc()

    def main(self):
        self.countOnly()
        self.oidsOnly()
        self.requestData()

if __name__ == '__main__':
    poo = RequestDataAGOL()
    poo.main()

In [ ]:
import os
from zipfile import ZipFile

files_to_zip = [x for x in os.listdir() if nombreSHP in x]
with ZipFile('{}.zip'.format(nombreSHP), 'w') as zipObj:
    for filename in files_to_zip:
        zipObj.write(filename, os.path.basename(filename))

In [ ]:
gdf = gpd.read_file("{}.shp".format(nombreSHP))
gdf.plot()

In [ ]:
len(gdf)

18000